In [1]:
import torch, dgl, accelerate
import torch.nn as nn
import torch.nn.functional as F
from accelerate.utils import set_seed
from dgl.distributed import partition_graph
import time
import numpy as np
import os
from tqdm import tqdm
import pandas as pd
import txgnn.TxGNN as TxGNN, txgnn.TxData as TxData, txgnn.TxEval as TxEval
import pickle

In [5]:
# seed = 1
# size = 100
# set_seed(seed)
# strt = time.time()
# saving_path = './properly_pre_trained_model_ckpt/'
# # split = 'cell_proliferation'

# txData = TxData(data_folder_path = './data/')
# split = 'complex_disease'
# txData.prepare_split(split=split, seed = seed, no_kg = False)
# txGNN = TxGNN(
#         data = txData, 
#         weight_bias_track = False,
#         proj_name = 'TxGNN',
#         exp_name = 'TxGNN',
#     )
# ## to load a pretrained model: 
# loading_path = './model_ckpt'

# txGNN.model_initialize(n_hid = size, 
#                       n_inp = size, 
#                       n_out = size, 
#                       proto = True,
#                       proto_num = 3,
#                       attention = False,
#                       sim_measure = 'all_nodes_profile',
#                       bert_measure = 'disease_name',
#                       agg_measure = 'rarity',
#                       num_walks = 200,
#                       walk_mode = 'bit',
#                       path_length = 2)

# TxGNN.load_pretrained(saving_path)
# # txGNN.save_model(f'{saving_path}seed_{seed}_normal')
# print("Done loading")
# print(time.time() - strt)

In [7]:
# file_path = 'psuedo_scores_all_indication.pkl'
# file_path = 'pscores_all_restrained_saveG_indication.pkl'
file_path = 'logs/eval_perf/soft_pseudo_inog_pneg_wog_e2000_0.95lr/59/0__indication.pkl'

with open(file_path, 'rb') as file:
    indication_results = pickle.load(file)
# indication_results

In [8]:
# file_path = 'pscores_all_restrained_saveG_contraindication.pkl'
file_path = 'logs/eval_perf/soft_pseudo_inog_pneg_wog_e2000_0.95lr/59/0__contraindication.pkl'

# file_path = 'psuedo_scores_all_contraindication.pkl'
with open(file_path, 'rb') as file:
    contraindication_results = pickle.load(file)
# contraindication_results

In [4]:
## load seed 1 train, val, test
df_train = pd.read_csv('data/complex_disease_1/train.csv', low_memory=False)
df_valid = pd.read_csv('data/complex_disease_1/valid.csv', low_memory=False)
df_test = pd.read_csv('data/complex_disease_1/test.csv', low_memory=False)
dd_df_train = df_train[(df_train.y_type == 'disease') & (df_train.x_type == 'drug')]
dd_df_valid = df_valid[(df_valid.y_type == 'disease') & (df_valid.x_type == 'drug')]
dd_df_test = df_test[(df_test.y_type == 'disease') & (df_test.x_type == 'drug')]
dd_df_all = pd.concat([dd_df_train, dd_df_valid, dd_df_test])

In [4]:
# seed = 1
# df_train = pd.read_csv(f'data/complex_disease_{seed}/train.csv', low_memory=False)
# df_valid = pd.read_csv(f'data/complex_disease_{seed}/valid.csv', low_memory=False)
# df_test = pd.read_csv(f'data/complex_disease_{seed}/test.csv', low_memory=False)
df = pd.concat([df_train, df_valid, df_test])
dd_df = df[(df.y_type == 'disease') & (df.x_type == 'drug')]
dd_df

,x_type,x_id,relation,y_type,y_id,x_idx,y_idx
3331331,drug,DB05271,contraindication,disease,5044.0,3448.0,12675.0
3331332,drug,DB05271,contraindication,disease,1200_1134_15512_5080_100078,3448.0,1569.0
3331333,drug,DB00492,indication,disease,5044.0,478.0,12675.0
3331334,drug,DB00492,indication,disease,1200_1134_15512_5080_100078,478.0,1569.0
3331335,drug,DB13956,contraindication,disease,5044.0,7534.0,12675.0
...,...,...,...,...,...,...,...
202019,drug,DB01152,indication,disease,6014.0,1129.0,13567.0
202020,drug,DB12612,contraindication,disease,20806.0,6808.0,7740.0
202021,drug,DB00137,indication,disease,2175.0,126.0,8307.0
202022,drug,DB09303,indication,disease,15905.0,5952.0,3739.0


In [14]:
import random
def turn_to_df(results, rel, k):
    # additional_train_dict = []
    # concat_additional_train_df = []
    concat_df_dd = []
    count = 0
    nested_concat_dfs = {"train": [], "valid": [], "test": []}
    for (dis_id, drug_ids), drug_idxs, dis_idx, ranked_scores in zip(results['ranked_drug_ids'].items(), results['ranked_drug_idxs'].values(), results['dis_idx'].values(), results['ranked_scores'].values()):
        new_dicts = [{'y_id': dis_id, 'y_idx': dis_idx, 'x_id': drug_id, 'x_idx': drug_idx, 'relation': rel, 'score': ranked_score} for i, (drug_id, drug_idx, ranked_score) in enumerate(zip(drug_ids, drug_idxs, ranked_scores))]
        
        # ## top and bottom k approach
        # top_k = new_dicts[:k]
        # bottom_k = new_dicts[-k:]
        # additional_train_dict += top_k + bottom_k

        ## Random Sampling
        # concat_additional_train_df.append(pd.DataFrame(random.sample(new_dicts, k)))

        ## in df_train, df_valid, or df_
        temp_df = pd.DataFrame(new_dicts)
        # temp_train = temp_df.merge(dd_df_train[['x_id', 'y_id', 'relation']], on=['x_id', 'y_id', 'relation'], how="inner")
        # temp_valid = temp_df.merge(dd_df_valid[['x_id', 'y_id', 'relation']], on=['x_id', 'y_id', 'relation'], how="inner")
        # temp_test = temp_df.merge(dd_df_test[['x_id', 'y_id', 'relation']], on=['x_id', 'y_id', 'relation'], how="inner")
        temp_dd_df = temp_df.merge(dd_df[['x_id', 'y_id', 'relation']], on=['x_id', 'y_id', 'relation'], how="inner")
        # nested_concat_dfs["train"].append(temp_train)
        # nested_concat_dfs["valid"].append(temp_valid)
        # nested_concat_dfs["test"].append(temp_test)
        concat_df_dd.append(temp_dd_df)

        # count += 1
        # if count == 10:
        #     break
        # if len(temp_train) != 0:
        #     break

    # for k, v in nested_concat_dfs.items():
    df = pd.concat(concat_df_dd)
    print(f"before dropping dup: {len(df)}")
    df = df.drop_duplicates()
    print(f"after dropping dup: {len(df)}")
    df["x_idx"] = df["x_idx"].astype(float)
    df["y_type"] = "disease"
    df["x_type"] = "drug"
        # nested_concat_dfs[k] = df
    return df
    # return nested_concat_dfs

In [15]:
k = 100
indication_dfs = turn_to_df(indication_results, "indication", k)
# contraindication_dfs = turn_to_df(contraindication_results, "contraindication", k)
# full_additional_df = pd.concat([indication_df, contraindication_df])

before dropping dup: 9388
after dropping dup: 9388


In [16]:
contraindication_dfs = turn_to_df(contraindication_results, "contraindication", k)

before dropping dup: 19255
after dropping dup: 19255


In [18]:
full_df_dd = pd.concat([indication_dfs, contraindication_dfs])

In [19]:
len(full_df_dd)

28643

In [8]:
full_additional_train = pd.concat([indication_dfs['train'], contraindication_dfs['train']])
full_additional_valid = pd.concat([indication_dfs['valid'], contraindication_dfs['valid']])
full_additional_test = pd.concat([indication_dfs['test'], contraindication_dfs['test']])

In [35]:
# full_additional_train.to_csv("toy_pseudo_scores_train.csv", index=False)
# full_additional_valid.to_csv("toy_pseudo_scores_valid.csv", index=False)
# full_additional_test.to_csv("toy_pseudo_scores_test.csv", index=False)

In [4]:
full_additional_train = pd.read_csv("toy_pseudo_scores_train.csv")
full_additional_valid = pd.read_csv("toy_pseudo_scores_valid.csv")
full_additional_test = pd.read_csv("toy_pseudo_scores_test.csv")

In [13]:
full_additional_valid[full_additional_valid.relation == 'indication'].score.sum()

1466.5144356716

In [12]:
len(full_additional_valid)
print(len(full_additional_valid[full_additional_valid.relation == 'indication']))

1039


In [10]:
print((torch.tensor(full_additional_train.score.values) > 0).float().mean())
print((torch.tensor(full_additional_valid.score.values) > 0).float().mean())
print((torch.tensor(full_additional_test.score.values) > 0).float().mean())

tensor(0.8583)
tensor(0.6598)
tensor(0.7385)


In [9]:
full_additional_df = pd.concat([full_additional_train, full_additional_valid, full_additional_test])
full_additional_df

,y_id,y_idx,x_id,x_idx,relation,score,y_type,x_type
0,12945_11691_12077_8780_11196_27694_11951_10459...,2026.0,DB00740,722.0,indication,1.421263,disease,drug
1,12945_11691_12077_8780_11196_27694_11951_10459...,2026.0,DB12243,6662.0,indication,-0.931736,disease,drug
0,10615_13006_8250_32569_9876_32567,621.0,DB00624,608.0,indication,2.631538,disease,drug
0,10619_10358_20720,624.0,DB14012,7567.0,indication,5.476806,disease,drug
1,10619_10358_20720,624.0,DB00153,142.0,indication,0.527278,disease,drug
...,...,...,...,...,...,...,...,...
0,17216.0,4875.0,DB02300,1837.0,contraindication,1.113359,disease,drug
1,17216.0,4875.0,DB06824,4133.0,contraindication,-0.157545,disease,drug
2,17216.0,4875.0,DB00443,429.0,contraindication,-0.826205,disease,drug
0,18136.0,5634.0,DB00553,537.0,contraindication,1.405244,disease,drug


In [10]:
print(len(dd_df_train))
print(len(full_additional_df))
print(len(dd_df_valid))
print(len(dd_df_test))

36324
40063
4664
1643


In [21]:
dd_df.to_csv("data/og_df_dd", index=False)

In [19]:
how = "left"
train_left = dd_df_train.merge(full_additional_train[['y_id', 'x_id', 'relation']], how=how, indicator=True)
train_left[train_left['_merge'] == 'left_only']

# full_additional_valid
# full_additional_test

,x_type,x_id,relation,y_type,y_id,x_idx,y_idx,_merge
288,drug,DB00903,off-label use,disease,5044.0,884.0,12675.0,left_only
289,drug,DB00903,off-label use,disease,1200_1134_15512_5080_100078,884.0,1569.0,left_only
568,drug,DB00887,off-label use,disease,5044.0,868.0,12675.0,left_only
569,drug,DB00887,off-label use,disease,1200_1134_15512_5080_100078,868.0,1569.0,left_only
823,drug,DB01235,off-label use,disease,5391.0,1211.0,12966.0,left_only
...,...,...,...,...,...,...,...,...
36247,drug,DB01045,off-label use,disease,41752.0,1024.0,11405.0,left_only
36266,drug,DB00410,off-label use,disease,6541.0,396.0,14081.0,left_only
36267,drug,DB00410,off-label use,disease,7554_7551_7555_7550_14661_9181_18747_15551_192...,396.0,14964.0,left_only
36268,drug,DB00410,off-label use,disease,1404.0,396.0,2546.0,left_only


In [ ]:
## Sub-questions
# Why are we missing 2000 pseudo scores?
# We saw improvement in training score at the least. Does this mean that the 2000 data are coming from valid and train only and causing the discrepency? 


In [92]:
# x_ids = indication_results['ranked_drug_idxs']['13924_12592_14672_13460_12591_12536_30861_8146_8148_32846_13459_44329_14544_9805_49223_9804_14086_8147_13515_14029_12581_19019']
# test = pd.DataFrame(x_ids, index=range(len(x_ids)), columns=['x_idx'])
# merged = dd_df_all[['x_idx']].merge(test[['x_idx']], on="x_idx", how='left', indicator=True)
# print(merged)
# merged[merged['_merge'] == "left_only"]

        x_idx _merge
0      3448.0   both
1      3448.0   both
2       478.0   both
3       478.0   both
4      7534.0   both
...       ...    ...
42626  1129.0   both
42627  6808.0   both
42628   126.0   both
42629  5952.0   both
42630  5951.0   both

[42631 rows x 2 columns]


,x_idx,_merge


In [ ]:
# 42631 - 40063
36324 - 34258

2066

In [27]:
# print(len(dd_df_train) + len(dd_df_valid) + len(dd_df_test) == len(full_additional_df))
print(len(dd_df_all.drop_duplicates()))
print(len(full_additional_df.drop_duplicates()))
# print(len(df_train_intersection) + len(df_valid_intersection) + len(df_test_intersection) == len(full_additional_df))


42631
40063


In [11]:
# full_additional_train.to_csv(f"psuedo_scores_top_and_bottom_{k}.csv", index=False)
full_additional_df.to_csv(f"psuedo_scores_restrained_inog_dataset.csv", index=False)

In [78]:
# full_additional_train = pd.read_csv('psuedo_scores_top_and_bottom_20.csv')

In [104]:
# df_train_intersection = full_additional_train.merge(dd_df_train, on=['y_id', 'x_id'], how='inner')
# (torch.tensor(indication_df.score.values) > 0).float().mean()
# (torch.tensor(contraindication_df.score.values) > 0).float().mean()
(torch.tensor(full_additional_df.score.values) > 0).float().mean()

tensor(0.8583)

In [46]:
full_additional_train.y_id.loc[0].values

array(['13924_12592_14672_13460_12591_12536_30861_8146_8148_32846_13459_44329_14544_9805_49223_9804_14086_8147_13515_14029_12581_19019',
       '13924_12592_14672_13460_12591_12536_30861_8146_8148_32846_13459_44329_14544_9805_49223_9804_14086_8147_13515_14029_12581_19019'],
      dtype=object)

In [48]:
full_additional_train[(full_additional_train.y_id == '13924_12592_14672_13460_12591_12536_30861_8146_8148_32846_13459_44329_14544_9805_49223_9804_14086_8147_13515_14029_12581_19019') & (full_additional_train.relation == "indication")]

,y_id,y_idx,x_id,x_idx,relation,score,y_type,x_type
0,13924_12592_14672_13460_12591_12536_30861_8146...,2502.0,DB00428,414.0,indication,4.515939,disease,drug
1,13924_12592_14672_13460_12591_12536_30861_8146...,2502.0,DB00233,221.0,indication,3.656111,disease,drug
2,13924_12592_14672_13460_12591_12536_30861_8146...,2502.0,DB06789,4106.0,indication,3.470831,disease,drug
3,13924_12592_14672_13460_12591_12536_30861_8146...,2502.0,DB06781,4098.0,indication,3.391865,disease,drug
4,13924_12592_14672_13460_12591_12536_30861_8146...,2502.0,DB00975,956.0,indication,3.315036,disease,drug
5,13924_12592_14672_13460_12591_12536_30861_8146...,2502.0,DB00635,619.0,indication,3.258958,disease,drug
6,13924_12592_14672_13460_12591_12536_30861_8146...,2502.0,DB04066,2760.0,indication,3.111462,disease,drug
7,13924_12592_14672_13460_12591_12536_30861_8146...,2502.0,DB02901,2153.0,indication,3.002678,disease,drug
8,13924_12592_14672_13460_12591_12536_30861_8146...,2502.0,DB12712,6842.0,indication,3.002594,disease,drug
9,13924_12592_14672_13460_12591_12536_30861_8146...,2502.0,DB00621,605.0,indication,2.992957,disease,drug
